## 2.15 字符串中插入变量

你想创建一个内嵌变量的字符串，变量被它的值所表示的字符串替换掉 

Python 并没有对在字符串中简单体寒变量值提供直接的支持。但是通过format()方法可以解决这个问题

In [12]:
s = '{name} has {num} message'

In [15]:
s.format(name='liyang',num='100')

'liyang has 100 message'

  或者，如果要被替换的标量能在变量域中找到，那么可以结合使用format_map()和vars()
  例如：

In [17]:
name = 'LiYang'
num = 100
s.format_map(vars())

'LiYang has 100 message'

vars()还有一个有意思的特性就是它也适用于对象实例，例如：

In [21]:
class Info:
    def __init__(self,name,num):
        self.name = name
        self.num = num

In [22]:
a = Info('new Liyang',102)
s.format_map(vars(a))

'new Liyang has 102 message'

format()和format_map()有一个缺陷就是他们不能很好的处理变量缺失的情况

一种避免这种错误的方法就是另外定义一个含有__missing__()方法的字典对象

In [25]:
class safesub(dict):
    """防止key找不到"""
    def __missing__(self,key):
        return '{'+ key + "}"

现在可以利用这个类包装后输入传递给format_map()函数，如果在变量域找不到key,则不会给变量复制

In [26]:
del num #确认num变量已经从变量域中删除

In [27]:
s.format_map(safesub(vars()))

'LiYang has {num} message'

In [32]:
import sys
def sub(text):
    return text.format_map(safesub(sys._getframe(1).f_locals))

In [36]:
name = 'New LIYANG'
num = 103

In [38]:
print(sub("{name} is so handsome"))

New LIYANG is so handsome


多年以来，由于Python缺乏对变量替换的内置支持而导致了各种不同的解决方案。相对来讲，format()和format_map() 相对于其他方案更加先进，因此应该优先被使用。
使用format()方法的另一个好处是，你可以获得对字符串格式化的所有支持（包括填充，对齐，数字格式化等等）而这些特性是使用其他方法所不能够做到的

对齐方式主要通过在字符串前使用 <  > 以及 ^ 来实现，详情可见2.13节

本节还介绍了python的一些高级特性。
映射或者字典中鲜为人知的 __missing__() 方法可以让你自定义如何处理缺失值。
在 SafeSub 类中，这个方法被定义为对缺失值返回一个占位符。你可以发现确实的值会出现在结果字符串中（在调试的时候可能会很有用），而不是产生一个KeyError异常

sub()函数使用 sys.__getframe(1) 返回调用者的栈帧，可以从中访问属性 f_locals 来获取局部变量

栈帧也叫活动记录器，是编译器用来实现过程/函数调用的一种数据结构。可以简单理解为：栈帧就是存储在用户栈上的（当然内核同样适用）每一次函数调用设计的相关信息的记录单元

 栈是从高地址向低地址延伸的，每个函数的每次调用，都有一个他自己独立的栈帧，这个栈帧中维持着所需要的各种信息。从低地址esp到高地址ebp的这块区域，就是当前函数的栈帧。
 栈帧表示程序的函数调用记录，而栈帧又是记录在栈上面的，很明显栈上保存了N个栈帧的实体，可以简单的理解为栈帧将栈分割成了N个记录块，每个记录块大小不是固定的，因为栈帧不仅保存诸如：函数入参、出参、返回地址和上一个栈帧的栈底指针(esp)信息，还保存了函数内部的自动变量

详细解释可以参考 https://blog.csdn.net/yxysdcl/article/details/5569351

毫无疑问绝大部分情况下在代码中区直接操作栈帧是不推荐的。但是，对于像字符串替换工具函数而言它是非常有用的，另外，值得注意的是 f_locals 是一个复制调用函数的本地变量的词典。这意味着你可以改变f_locals的内容，但是这个修改对于后面的变量访问没有任何影响。因此，虽说方位一个栈帧看上去很邪恶，但是对他的任何操作不会覆盖和改变调用者本地变量的值